# Databricks Unity Catalog - Table ACL

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-0.png?raw=true" style="float: right" width="500px"/> 

The main feature of Unity Catalog is to provide you an easy way to setup Table ACL (Access Control Level), but also build Dynamic Views based on each individual permission.

Typically, Analysts will only have access to customers from their country and won't be able to read GDPR/Sensitive informations (like email, firstname etc.)

A typical workflow in the Lakehouse architecture is the following:

* Data Engineers / Jobs can read and update the main data/schemas (ETL part)
* Data Scientists can read the final tables and update their features tables
* Data Analyst have READ access to the Data Engineering and Feature Tables and can ingest/transform additional data in a separate schema.
* Data is masked/anonymized dynamically based on each user access level

With Unity Catalog, your tables, users and groups are defined at the account level, cross workspaces. Ideal to deploy and operate a Lakehouse Platform across all your teams.

In [0]:
%run ./_resources/00-setup

## Creating the CATALOG

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/uc/uc-base-1.png?raw=true" style="float: right" width="800px"/> 

Unity Catalog works with 3 layers:

* CATALOG
* SCHEMA (or DATABASE)
* TABLE

To access one table, you can specify the full path: `SELECT * FROM &lt;CATALOG&gt;.&lt;SCHEMA&gt;.&lt;TABLE&gt;`

Note that the tables created before Unity Catalog are saved under the catalog named `hive_metastore`. Unity Catalog features are not available for this catalog.

In [0]:
CREATE CATALOG IF NOT EXISTS uc_acl_learning
   MANAGED LOCATION 'abfss://unity-catalog-storage@dbstoragefuavvebhyzyxm.dfs.core.windows.net/3759185753378633'

In [0]:
USE CATALOG uc_acl_learning;

SELECT CURRENT_CATALOG();

## Creating the SCHEMA
Next, we need to create the SCHEMA (or DATABASE).

In [0]:
CREATE SCHEMA IF NOT EXISTS dev;
USE SCHEMA dev;

## Creating our table

We're all set! We can use standard SQL to create our tables. We'll use a customers dataset, loading data about users (id, email etc...)

Because we want our demo to be available for all, we'll grant full privilege to the table to all USERS.

Note that the table owner is the current user. Owners have full permissions.<br/>
If you want to change the owner you can set it as following: ```ALTER TABLE <catalog>.dev.customers OWNER TO `account users`;```

In [0]:
CREATE TABLE IF NOT EXISTS customers (
  id STRING,
  creation_date STRING,
  firstname STRING,
  lastname STRING,
  country STRING,
  email STRING,
  address STRING,
  gender DOUBLE,
  age_group DOUBLE
);

In [0]:
-- For the demo only, allow all users to edit the table.
GRANT SELECT, MODIFY ON TABLE customers TO `account users`;
SHOW GRANTS ON TABLE customers;

In [0]:
SELECT * FROM customers;

## Granting users or group access

Unity catalog let you GRANT standard SQL permission to your objects, using the Unity Catalog users or group.

### Creating groups

Databricks groups can be created at the account level using the Account Admin UI, or the SCIM API. Here, we created the `dataengineers` group for this demo.

*Note on workspace-level groups: you can also create groups at a workspace level, however, we recommend managing permissions with UC at an account level.*

In [0]:
-- We'll grant an extra MODIFY to our Data Engineer
-- Note: make sure you created the dataengineers group first as an account admin!
GRANT SELECT, MODIFY ON TABLE customers TO `dataengineers`;

In [0]:
SHOW GRANTS ON TABLE customers;